In [1]:
%cd /content
import os
import sys
from google.colab import drive
import cv2
import numpy as np
import matplotlib.pyplot as plt
from google.colab import drive
import math
from skimage.draw import polygon
from skimage.feature import peak_local_max
import torch.nn.functional as F
import json
drive.mount('/content/gdrive')
sys.path.append('/content/Yolact-pytorch')
!git clone https://github.com/ARG-NCTU/Yolact-pytorch.git
!pip install tensorboardX
!pip install terminaltables

/content
Mounted at /content/gdrive
Cloning into 'Yolact-pytorch'...
remote: Enumerating objects: 444, done.
remote: Counting objects: 100% (444/444), done.
remote: Compressing objects: 100% (315/315), done.
remote: Total 444 (delta 144), reused 391 (delta 100), pack-reused 0
Receiving objects: 100% (444/444), 12.85 MiB | 24.05 MiB/s, done.
Resolving deltas: 100% (144/144), done.
     |████████████████████████████████| 317kB 7.4MB/s 
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp36-none-any.whl size=15358 sha256=53cd21ca2f13d005f82bec150ed1cf470df4546ba2a73c2b37d13ad734f19708
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built terminaltables


In [2]:
import sys
import numpy as np
import cv2
import matplotlib.pyplot as plt
import struct
import time
import os
import math
import argparse

import PIL
import pandas as pd
import scipy.misc
import random
# from cv_bridge import CvBridge, CvBridgeError
# from std_msgs.msg import Header
from datetime import datetime

from torchvision import transforms, utils, datasets
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch
from torchvision import models
from torchvision.models.vgg import VGG
# from sklearn.metrics import confusion_matrix
# from subt_msgs.msg import *

import torch.backends.cudnn as cudnn
from modules.build_yolact import Yolact
from utils.augmentations import FastBaseTransform
from utils.functions import MovingAverage, ProgressBar
from utils import timer
from data.config import update_config, COLORS, cfg
from utils.output_utils import NMS, after_nms, draw_img

import pyximport
pyximport.install(setup_args={"include_dirs": np.get_include()}, reload_support=True)

/usr/local/lib/python3.6/dist-packages/Cython/Compiler/Main.py:369: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /content/Yolact-pytorch/utils/cython_nms.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)


(None, None)

In [3]:
import os
import sys
import logging
import gdown
from zipfile import ZipFile
%cd /content
dataset_url = "https://drive.google.com/u/1/uc?id=1OR36Mmu252_rkUo9bEgk4tx71J2ZwbGd&export=download"
dataset_name = "2_latest_res101_custom_800001"
if not os.path.isdir(dataset_name):
    gdown.download(dataset_url, output=dataset_name + '.pth', quiet=False)

print("Finished downloading dataset.")

/content


Downloading...
From: https://drive.google.com/u/1/uc?id=1OR36Mmu252_rkUo9bEgk4tx71J2ZwbGd&export=download
To: /content/2_latest_res101_custom_800001.pth
199MB [00:02, 91.6MB/s]


Finished downloading dataset.


In [4]:
model_name = '/content/2_latest_res101_custom_800001.pth'
strs = model_name.split('_')
config = strs[-3] + "_" + strs[-2] + "_config"
update_config(config)
cuda = torch.cuda.is_available()
if cuda:
  cudnn.benchmark = True
  cudnn.fastest = True
  torch.set_default_tensor_type('torch.cuda.FloatTensor')
else:
  torch.set_default_tensor_type('torch.FloatTensor')

net = Yolact()
net.load_weights(model_name, cuda)
net.cuda()

Yolact(
  (backbone): ResNetBackbone(
    (layers): ModuleList(
      (0): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu): ReLU(inplace=True)
          (downsample): Sequential(
            (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          )
        )
        (1): Bottleneck(
          (conv1): Conv2d(256, 64, ker

In [5]:
parser = argparse.ArgumentParser(description='YOLACT Predict in ROS')
parser.add_argument('--visual_top_k', default=100, type=int, help='Further restrict the number of predictions to parse')
parser.add_argument('--traditional_nms', default=False, action='store_true', help='Whether to use traditional nms.')
parser.add_argument('--hide_mask', default=False, action='store_true', help='Whether to display masks')
parser.add_argument('--hide_bbox', default=True, action='store_true', help='Whether to display bboxes')
parser.add_argument('--hide_score', default=True, action='store_true', help='Whether to display scores')
parser.add_argument('--show_lincomb', default=False, action='store_true',
							help='Whether to show the generating process of masks.')
parser.add_argument('--no_crop', default=False, action='store_true',
							help='Do not crop output masks with the predicted bounding box.')
parser.add_argument('--real_time', default=True, action='store_true', help='Show the detection results real-timely.')
parser.add_argument('--visual_thre', default=0.3, type=float,help='Detections with a score under this threshold will be removed.')
args = parser.parse_args(args=['--visual_top_k', '100','--traditional_nms','--hide_mask','--hide_bbox','--show_lincomb','--no_crop','--real_time','--visual_thre','0.3'])

In [6]:
args.hide_mask = False
args.no_crap = False
args.traditional_nms = False
args.hide_bbox = True

In [7]:
def predictor(img):
  rgb_origin = img
  img_numpy = img

  img = torch.from_numpy(img.copy()).float()
  img = img.cuda()

  img_h, img_w = img.shape[0], img.shape[1]
  img_trans = FastBaseTransform()(img.unsqueeze(0))

  net_outs = net(img_trans)
  nms_outs = NMS(net_outs, 0)

  results = after_nms(nms_outs, img_h, img_w, crop_masks=not args.no_crop, visual_thre=args.visual_thre)
  # print(results)
  masks = results[3][:args.visual_top_k]
  masks = masks.cpu().detach().numpy()
  cls = results[0][:args.visual_top_k]
  cls = cls.cpu().detach().numpy()
  torch.cuda.synchronize()
  frame_numpy = draw_img(results, img, args, class_color=True, fps=0)
  # print(cls)

  return frame_numpy,masks,cls

In [8]:
def mask2mask(mask):
  canvas  =  np.zeros((mask.shape[0], mask.shape[1], 3), dtype = "uint8")
  for i in range(mask.shape[0]):
    for j in range(mask.shape[1]):
      if mask[i,j] == True:
          canvas[i,j] = 255

  gray = cv2.cvtColor(canvas,cv2.COLOR_BGR2GRAY)  
  ret, binary = cv2.threshold(gray,127,255,cv2.THRESH_BINARY)
  contours, hierarchy = cv2.findContours(binary,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
  # area = cv2.contourArea(np.array(contours)[0])
  # print(area)
  # print(len(contours))
  if len(contours) != 0:
    tt = np.array(contours)[0]
    polygon = np.reshape(tt,(tt.shape[0],2))
  else:
    polygon = []
  # print(np.array(contours)[0].shape)
  # polygon = np.reshape(tt,(tt.shape[0],2))
  return polygon

In [9]:
def json2polygon(path):
    classes = 0
    with open(path,"r") as f:
      data = json.load(f)
      polygon = np.array(data['shapes'][0]['points'],dtype=np.int32)
      label = data['shapes'][0]['label']
      if label == 'backpack':
        classes = 0
      if label == 'survivor':
        classes = 3
      if label == 'vent':
        classes = 1
      if label == 'phone':
        classes = 2
      

    return polygon,classes

In [10]:
def draw_polygon(polygon__):
  # canvas  =  np.zeros((480, 640, 3), dtype = "uint8")
  canvas  =  np.zeros((544, 1024, 3), dtype = "uint8")
  points  =  polygon__.reshape((-1, 1, 2))
  cv2.fillPoly(canvas,[points],(255,255,255))
  # plt.figure(figsize=(15,15))
  plt.imshow(canvas)
  plt.show()

In [11]:
def polygon_IOU(polygon_1, polygon_2):
    """
    count iou of two polygon
    :param polygon_1: [[row1, col1], [row2, col2], ...]
    :param polygon_2: same as above
    :return:
    """
    rr1, cc1 = polygon(polygon_2[:, 0], polygon_2[:, 1])
    rr2, cc2 = polygon(polygon_1[:, 0], polygon_1[:, 1])

    try:
        r_max = max(rr1.max(), rr2.max()) + 1
        c_max = max(cc1.max(), cc2.max()) + 1
    except:
        return 0

    canvas = np.zeros((r_max, c_max))
    canvas[rr1, cc1] += 1
    canvas[rr2, cc2] += 1
    union = np.sum(canvas > 0)
    if union == 0:
        return 0
    intersection = np.sum(canvas == 2)

    return intersection / union

In [30]:
def Image_load(directory_name):
    image_list = []
    directory_name = directory_name + '/image'
    path = os.listdir(directory_name)
    path.sort(key=lambda x:int(x[14:-4]))
    f = 0
    for filename in path:
        #print(filename)
        if f <219:
          img = cv2.imread(directory_name + "/" + filename)
          image_list.append(img)
        f += 1
        
    return image_list

In [34]:
def Label_load(directory_name):
    mask_list = []
    classes_list = []
    directory_name = directory_name + '/json'
    path = os.listdir(directory_name)
    path.sort(key=lambda x:int(x[14:-5]))
    f = 0
    for filename in path:
        #print(filename)
      if f <219:
        mask,cls = json2polygon(directory_name + "/" + filename)
        mask_list.append(mask)
        classes_list.append(cls)
      f += 1

    return mask_list, classes_list

In [20]:
Path = '/content/gdrive/MyDrive/Artifact/ex_B_1/ex_B1_survivor_708'

In [32]:
image_list = Image_load(Path)

In [35]:
truemask_list, cls_list = Label_load(Path)

In [36]:
len(truemask_list)

219

In [ ]:
# cls_list

In [37]:
count = 0
for i in range(len(truemask_list)):
  h = i
  print('Image : ',h)
  # h = i+641
  temp = []
  mask_temp = []
  get = 0
  predict_img,Mask,classes = predictor(image_list[h])
  for j in range(Mask.shape[0]):
      if classes[j] == cls_list[h]:
        pred = mask2mask(Mask[j])
        if len(pred) != 0:
          # mask_temp.append(pred)
          get += 1
          iou = polygon_IOU(truemask_list[h], pred)
          temp.append(iou)

  plt.imshow(predict_img[:,:,[2,1,0]])
  plt.show()
  # if (h >= 641) and (h<842):
  #       # cv2.imwrite('/content/t/true_'+str(h)+'.jpg',image_list[h])
  cv2.imwrite('/content/gdrive/MyDrive/s_yolact/yolact_'+str(h)+'.jpg',predict_img)
  if get != 0:
     index = temp.index(max(temp))
    #  print('Mask')
    #  draw_polygon(mask_temp[index])
     print('Label')
     draw_polygon(truemask_list[h])
     if temp[index] != 0.0:
      print('IOU : ',temp[index])
      if temp[index] >= 0.5:
        count += 1
     else:
      print('No detect')

  else:
     draw_polygon(truemask_list[h])
     print('No detect')

  print('Detect : ',count,' / ',len(truemask_list))
  print('=================')

Output hidden; open in https://colab.research.google.com to view.